In [2]:
import logging
import tempfile

import gym
import numpy as np
import pandas as pd
from gym import spaces
from gym.utils import seeding
from sklearn.preprocessing import scale
import talib


In [7]:
def load_data():
        idx = pd.IndexSlice
        with pd.HDFStore('/Users/pepo_abdo/Desktop/ml/trad_ml/data/assets.h5') as store:
            df = (store['quandl/wiki/prices']
                  .loc[idx[:, 'AAPL'],
                       ['adj_close', 'adj_volume', 'adj_low', 'adj_high']]
                  .dropna()
                  .sort_index())
        df.columns = ['close', 'volume', 'low', 'high']
        return df

In [14]:
df = load_data()

In [15]:
df.head()

,,close,volume,low,high
date,ticker,,,,
1980-12-12,AAPL,0.422706,117258400.0,0.422706,0.424470
1980-12-15,AAPL,0.400652,43971200.0,0.400652,0.402563
1980-12-16,AAPL,0.371246,26432000.0,0.371246,0.373010
1980-12-17,AAPL,0.380362,21610400.0,0.380362,0.382273
1980-12-18,AAPL,0.391536,18362400.0,0.391536,0.393300


In [38]:
def preprocess_data(data):
        """calculate returns and percentiles, then removes missing values"""

        data['returns'] = data.close.pct_change()
        data['ret_2'] = data.close.pct_change(2)
        data['ret_5'] = data.close.pct_change(5)
        data['ret_10'] = data.close.pct_change(10)
        data['ret_21'] = data.close.pct_change(21)
        data['rsi'] = talib.STOCHRSI(data.close)[1]
        data['macd'] = talib.MACD(data.close)[1]
        data['atr'] = talib.ATR(data.high, data.low, data.close)

        slowk, slowd = talib.STOCH(data.high, data.low, data.close)
        data['stoch'] = slowd - slowk
        data['atr'] = talib.ATR(data.high, data.low, data.close)
        data['ultosc'] = talib.ULTOSC(data.high, data.low, data.close)
        data = (data.replace((np.inf, -np.inf), np.nan)
                     .drop(['high', 'low', 'close', 'volume'], axis=1)
                     .dropna())

        r = data.returns.copy()
        if True:
            data = pd.DataFrame(scale(data),
                                     columns=data.columns,
                                     index=data.index)
        features = data.columns.drop('returns')
        data['returns'] = r  # don't scale returns
        data = data.loc[:, ['returns'] + list(features)]
        log.info(data.info())
        return data

In [39]:
data = preprocess_data(df)

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9367 entries, (Timestamp('1981-01-30 00:00:00'), 'AAPL') to (Timestamp('2018-03-27 00:00:00'), 'AAPL')
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   returns  9367 non-null   float64
 1   ret_2    9367 non-null   float64
 2   ret_5    9367 non-null   float64
 3   ret_10   9367 non-null   float64
 4   ret_21   9367 non-null   float64
 5   rsi      9367 non-null   float64
 6   macd     9367 non-null   float64
 7   atr      9367 non-null   float64
 8   stoch    9367 non-null   float64
 9   ultosc   9367 non-null   float64
dtypes: float64(10)
memory usage: 1.5+ MB


In [37]:
data.head(5)

,,returns,ret_2,ret_5,ret_10,ret_21,rsi,macd,atr,stoch,ultosc
date,ticker,,,,,,,,,,
1981-01-30,AAPL,-0.054235,-0.088710,-0.137405,-0.088710,-0.172282,-7.401487e-15,0.005886,0.014674,3.288889e+00,19.338480
1981-02-02,AAPL,-0.057345,-0.108470,-0.174264,-0.189839,-0.228116,-7.401487e-15,0.003437,0.015328,-1.203975e-14,14.341183
1981-02-03,AAPL,0.037176,-0.022301,-0.136875,-0.133626,-0.181630,1.049348e+01,0.000924,0.015409,-4.899777e+00,23.392601
1981-02-04,AAPL,0.036206,0.074728,-0.076774,-0.119385,-0.112558,3.984699e+01,-0.001264,0.015495,-1.557221e+01,30.464308
1981-02-05,AAPL,0.000000,0.036206,-0.041848,-0.129297,-0.073187,7.318032e+01,-0.003132,0.014651,-2.630322e+01,29.407573


In [40]:
data.head(5)

,,returns,ret_2,ret_5,ret_10,ret_21,rsi,macd,atr,stoch,ultosc
date,ticker,,,,,,,,,,
1981-01-30,AAPL,-0.054235,-2.182875,-2.203772,-1.085791,-1.447542,-1.529044,-0.171568,-0.601086,0.282942,-2.763728
1981-02-02,AAPL,-0.057345,-2.657618,-2.773044,-2.191345,-1.862101,-1.529044,-0.174969,-0.600210,0.000242,-3.206694
1981-02-03,AAPL,0.037176,-0.587404,-2.195593,-1.576822,-1.516944,-1.211224,-0.178460,-0.600101,-0.420924,-2.404365
1981-02-04,AAPL,0.036206,1.743708,-1.267360,-1.421133,-1.004094,-0.322182,-0.181500,-0.599986,-1.338284,-1.777520
1981-02-05,AAPL,0.000000,0.818215,-0.727939,-1.529499,-0.711763,0.687397,-0.184094,-0.601118,-2.260681,-1.871191


In [43]:
trading_days=252

In [70]:
def reset(data):
        """Provides starting index for time series and resets step"""
        high = len(data.index) - trading_days
        offset = np.random.randint(low=0, high=high)
        step = 0
        return offset , step

def take_step(data,offset,step):
        """Returns data for current trading day and done signal"""
        obs = data.iloc[offset + step].values
        step += 1
        done = step > trading_days
        return obs, done

In [71]:
offset , step = reset(data)

In [72]:
steps_walk = take_step(data , offset,step)

In [73]:
steps_walk

(array([ 0.05048335, -0.87025527, -1.47521871, -0.00877511,  0.00250678,
        -1.19554507, -0.15901278, -0.50860528,  1.17282362,  0.69249948]),
 False)

In the context of investing and trading, "market_navs" and "nav" typically refer to financial metrics related to investment funds, such as mutual funds or exchange-traded funds (ETFs). Let's break down what each of these terms means and then provide an example using AAPL (Apple Inc.) to illustrate them in depth.

# NAV (Net Asset Value):
- What is NAV: NAV is the total value of the assets held by an investment fund (e.g., a mutual fund or ETF) minus any liabilities. It represents the per-share or per-unit value of the fund.
Calculation: NAV per share (or unit) = (Total Asset Value - Total Liabilities) / Total Number of Shares (or Units Outstanding).
# Market_navs (Market Net Asset Value):
- What is Market_navs: Market_navs is similar to NAV but takes into account the current market prices of the assets held by the fund. It's essentially the NAV adjusted for market fluctuations.

Calculation: Market_navs per share (or unit) = (Total Market Value of Assets - Total Liabilities) / Total Number of Shares (or Units Outstanding).
Example using AAPL (Apple Inc.):

Suppose you are interested in buying or selling shares in a mutual fund or ETF that holds AAPL stock as one of its assets. Here's how "NAV" and "Market_navs" would apply:

- NAV Calculation:
Let's assume the fund holds assets worth $10 million, has liabilities of $2 million, and has 1 million shares outstanding.
NAV per share = ($10,000,000 - $2,000,000) / 1,000,000 = $8 per share.

- Market_navs Calculation:
To calculate Market_navs, you need to consider the current market prices of the assets, including AAPL shares. Let's say the fund holds 100,000 AAPL shares, and AAPL's current market price is $150 per share.
Total Market Value of AAPL shares = 100,000 shares * $150/share = $15,000,000.
Total Market Value of Other Assets = $10,000,000 (assuming they haven't changed).
Total Market_navs = ($15,000,000 + $10,000,000 - $2,000,000) / 1,000,000 = $23 per share.
Now, let's consider buying or selling shares in this fund:

If you want to buy shares in this fund, you would typically buy them at the NAV price of $8 per share. This is the price at which the fund values its assets minus liabilities.
If you want to sell shares in this fund, you would typically sell them at the Market_navs price of $23 per share. This price reflects the current market values of the assets in the fund.
In summary, NAV represents the intrinsic value of the fund, while Market_navs takes into account the current market values of its assets. When buying or selling, you would typically use Market_navs as it reflects the real-time market conditions.

1. Strategy Returns:
- What are Strategy Returns: Strategy returns refer to the gains or losses generated by a specific trading or investment strategy over a particular period.
Use Example: If you have an investment strategy that involves buying and selling stocks, the strategy returns would reflect the overall profitability of your strategy after accounting for gains and losses on your positions.
2. Positions:
- What are Positions: Positions represent the securities (stocks, bonds, options, etc.) that an investor or trader currently holds in their portfolio. Each position includes details like the quantity of the security and its current market value.
Use Example: If you own 100 shares of AAPL and 50 shares of GOOGL, these holdings represent your positions in those stocks.
3. Costs:
- What are Costs: Costs in trading and investing refer to expenses associated with executing and managing your portfolio. This can include trading commissions, fees, taxes, and other transaction costs.
Use Example: If you paid $10 in trading commissions when buying or selling stocks, that $10 would be considered a  cost.
4. Trades:
- What are Trades: Trades represent the buying or selling of securities in the financial markets. Each trade includes details such as the security traded, the quantity, the price at which the trade was executed, and the time of the trade.
Use Example: If you bought 100 shares of XYZ stock at $50 per share, that transaction would be recorded as a trade.
5. Market Returns:
- What are Market Returns: Market returns indicate the overall performance of the financial markets or a specific market index over a period of time. It reflects how the prices of securities within that market have changed.
Use Example: If the S&P 500 index increased by 10% over a year, that would be the market return for the S&P 500 during that period.
In a trading or investment context, these terms are interrelated:

- Positions represent what you currently own.
- Trades are the actions you take to adjust your positions (buying or selling).
- Costs represent the expenses associated with these trades.
- Market Returns provide a benchmark for assessing how the broader market is performing.
- Strategy Returns evaluate the performance of your specific investment or trading strategy, often in comparison to market returns.

To calculate the Net Asset Values (NAVs), trading costs, and rewards based on an action and the latest market return, you can follow these steps:

1. Get the Current Portfolio's Positions and NAV:
- Retrieve the current positions (holdings) in your portfolio, including the quantity and market value of each security.
Calculate the Net Asset Value (NAV) of your portfolio. This is the total value of all your holdings.
2. Execute the Action:
- Execute the specified action (buy or sell) for a particular security based on your trading strategy.
Record the trade details, including the quantity, price, and any applicable trading costs (commissions, fees, taxes).
3. Update the Portfolio Positions and NAV:
- Adjust your portfolio positions and NAV based on the executed action and trade details. For a buy action, increase the position; for a sell action, decrease the position.
4. Calculate the Trading Costs:
- Calculate the trading costs associated with the executed trade. This includes any commissions or fees incurred during the trade.
5. Get the Latest Market Return:
- Retrieve the latest market return for the relevant benchmark index or asset. This represents how the market performed on that day.
6. Calculate the Portfolio's Daily Return:
- Calculate the daily return of your portfolio based on the changes in the NAV before and after the action.
7. Calculate the Reward:
- Calculate the reward or performance measure for your portfolio. This could be a simple difference between your portfolio's return and the market return or a more complex metric based on your strategy.
8. Generate a Summary of the Day's Activity:
- Create a summary report that includes:
- Details of the action taken (buy/sell).
- Trade details (quantity, price, costs).
- Updated portfolio positions and NAV.
- Daily portfolio return.
- Trading costs.
- Market return.
- Reward or performance measure.

In [74]:
# Sample data
current_positions = {'AAPL': {'quantity': 100, 'market_value': 15000}}
market_return = 0.01  # Assume a 1% market return
action = 'buy'
action_security = 'AAPL'
action_quantity = 50
action_price = 160  # Assuming $160 per share
commission_cost = 10  # Trading commission

# Calculate NAV
current_nav = sum(position['market_value'] for position in current_positions.values())

# Execute the action and update positions
if action == 'buy':
    current_positions[action_security]['quantity'] += action_quantity
    current_positions[action_security]['market_value'] += action_quantity * action_price
elif action == 'sell':
    current_positions[action_security]['quantity'] -= action_quantity
    current_positions[action_security]['market_value'] -= action_quantity * action_price

# Calculate trading costs
trading_costs = commission_cost

# Calculate daily return
previous_nav = current_nav - (action_quantity * action_price) if action == 'buy' else current_nav + (action_quantity * action_price)
daily_return = (current_nav - previous_nav) / previous_nav

# Calculate the reward
reward = daily_return - market_return

# Generate a summary report
summary = {
    'Action': action,
    'Trade Details': {'Security': action_security, 'Quantity': action_quantity, 'Price': action_price, 'Costs': trading_costs},
    'Updated Positions': current_positions,
    'Daily Return': daily_return,
    'Trading Costs': trading_costs,
    'Market Return': market_return,
    'Reward': reward
}

# Print the summary report
print(summary)


{'Action': 'buy', 'Trade Details': {'Security': 'AAPL', 'Quantity': 50, 'Price': 160, 'Costs': 10}, 'Updated Positions': {'AAPL': {'quantity': 150, 'market_value': 23000}}, 'Daily Return': 1.1428571428571428, 'Trading Costs': 10, 'Market Return': 0.01, 'Reward': 1.1328571428571428}
